In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [55]:
n_inputs = 28*28 # mnist
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = 'X')
y = tf.placeholder(tf.int64, shape = (None), name = 'y')

In [56]:
def neuron_layer(X, n_neurons, name,activation = None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev = stddev)
        W = tf.Variable(init, name = 'weights')
        b = tf.Variable(tf.zeros([n_neurons]), name = 'biases')
        z = tf.matmul(X, W)+b
        if activation == 'relu':
            return tf.nn.relu(z)
        else:
            return z

In [57]:
import numpy as np
with tf.name_scope('dnn'):
    hidden1 = neuron_layer(X, n_hidden1, 'hidden1', activation = 'relu')
    hidden2 = neuron_layer(hidden1, n_hidden2, 'hidden2', activation = 'relu')
    logits = neuron_layer(hidden2, n_outputs, 'outputs')
    print(logits.shape)

(?, 10)


In [58]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
    loss = tf.reduce_mean(xentropy, name = 'loss')

In [59]:
learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    trainin_op = optimizer.minimize(loss)
    

In [61]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [62]:
#!git clone https://github.com/tensorflow/tensorflow.git

In [63]:
import os
os.getcwd()
os.chdir('/home/odemakinde/Desktop/miscellaneous/miscellaneous/Convolutional Neural networks/tensorflow/tensorflow/examples/tutorials/mnist')
import input_data
mnist= input_data.read_data_sets('MNIST_data/', one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [70]:
def arg_max(data):
    arr = np.array([np.argmax(i) for i in data])
    return arr

In [83]:
n_epochs = 5
batch_size = 50

    

with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(trainin_op, feed_dict = {X:x_batch, y:arg_max(y_batch)})
        acc_train = accuracy.eval(feed_dict ={X:x_batch, y:arg_max(y_batch)})
        acc_test = accuracy.eval(feed_dict={X:mnist.test.images, 
                                           y:arg_max(mnist.test.labels)})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    
    save_path = saver.save(sess, ".my_modelfinal.ckpt")
    writer.close()

0 Train accuracy: 0.94 Test accuracy: 0.9132
1 Train accuracy: 1.0 Test accuracy: 0.9272
2 Train accuracy: 0.94 Test accuracy: 0.9374
3 Train accuracy: 1.0 Test accuracy: 0.9431
4 Train accuracy: 1.0 Test accuracy: 0.9476


In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./my_modelfinal.ckpt")
    x_new_scaled = [...] # some new images
    z = logits.eval(feed_dict = {X:X_new_scaled})
    y_pred = np.argmax(z, axis = 1)
    

In [84]:
import datetime, os
import tensorboard
#%load_ext tensorboard

%reload_ext tensorboard
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir} --host localhost --port 8088

Reusing TensorBoard on port 8088 (pid 20249), started 0:02:54 ago. (Use '!kill 20249' to kill it.)

In [ ]:
# neural network hyperparameter tuining.

